In [1]:
import pandas as pd

# Load BLAST results
cols = ["qseqid", "sseqid", "pident", "length", "qlen", "slen", "evalue", "bitscore"]
df = pd.read_csv("/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/24.Transcriptome_comparisons/1.Baker_et_al_2009_Gene_expression_during/blast_results_Diapause induced gene table.txt", sep="\t", names=cols)

# Compute query coverage
df["qcov"] = df["length"] / df["qlen"] * 100

# Filter: good e-value & coverage
filtered = df[(df["evalue"] < 1e-5) & (df["qcov"] > 50)]

# Sort by query, then best bitscore (or lowest evalue)
filtered = filtered.sort_values(["qseqid", "evalue", "bitscore"], ascending=[True, True, False])

# Pick top hit per query
top_hits = filtered.groupby("qseqid").first().reset_index()

# Save results
top_hits.to_csv("/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/24.Transcriptome_comparisons/1.Baker_et_al_2009_Gene_expression_during/top_hits_blast_results_Diapause induced gene table.tsv", sep="\t", index=False)
print(top_hits)

          qseqid       sseqid  pident  length  qlen  slen         evalue  \
0    FBgn0000056  10359at7088  27.354     223   272   253   2.300000e-08   
1    FBgn0000092  10750at7088  27.511     229   260   234   2.660000e-25   
2    FBgn0000299    964at7088  42.325    1798  1779  1758   0.000000e+00   
3    FBgn0000570   7277at7088  56.948     367   483   359  1.580000e-137   
4    FBgn0000635    496at7088  23.944     497   873   897   6.730000e-18   
..           ...          ...     ...     ...   ...   ...            ...   
263  FBgn0259149   4371at7088  28.641     412   521   725   8.950000e-48   
264  FBgn0259164   3678at7088  33.747     403   662   486   4.490000e-64   
265  FBgn0259175   2324at7088  35.628     741   784   824  1.300000e-149   
266  FBgn0259211   1253at7088  54.973     744  1333   701   0.000000e+00   
267  FBgn0259680   1372at7088  70.686     423   671  1333   0.000000e+00   

     bitscore        qcov  
0        51.2   81.985294  
1        97.8   88.076923  
2  

In [ ]:
rho_dict = {}
rho_genes = []
negative_rho_genes = []
with open("/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/1.Rerconverge_result/correlation_annotated.csv",'r') as rho_file:
    for lines in rho_file.readlines()[1:]:
        lines_split = lines.strip().split(",")
        
        # print(lines_split)
        # assert False
        gene = (lines_split[0][1:-1].split(".")[0])
        if float(lines_split[1]) > 0 and float(lines_split[5]) < 0.1:
            rho_genes.append(gene)
        if float(lines_split[1]) < 0 and float(lines_split[5]) < 0.1:
            negative_rho_genes.append(gene)
        rho_dict[gene] = [round(float(lines_split[1]),3),lines_split[-1]]
        # # print(rho_dict)
        # # assert False

print(rho_dict)
print(rho_genes,"\n", len(rho_genes), len(negative_rho_genes))


extended_rho_set = []
with open("/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/12.Rho_networks/2.Diapausing/Filtered_set_core_set.csv",'r') as extended_rho_file:
    for lines in extended_rho_file.readlines()[1:]:
        lines_split = lines.strip().split(",")
        if lines_split[0] not in extended_rho_set:
            extended_rho_set.append(lines_split[0])
        if lines_split[1] not in extended_rho_set:
            extended_rho_set.append(lines_split[1])
print(len(extended_rho_set))


        

{'clock': [0.216, 'clock'], 'cry1a': [0.114, 'cry1a'], 'cry1b': [0.032, 'cry1b'], 'cry2': [0.16, 'cry2'], 'cwo': [0.179, 'cwo'], 'cycle': [0.246, 'cycle'], 'period': [0.303, 'period'], 'timeless': [0.171, 'timeless'], 'timeout': [-0.132, 'timeout'], '1058at7088': [0.028, 'COMP'], '13199at7088': [0.119, '13199at7088'], '13230at7088': [-0.029, '13230at7088'], '13527at7088': [0.13, 'ALYREF'], '13532at7088': [-0.128, '13532at7088'], '137at7088': [-0.262, 'sev'], '14013at7088': [-0.01, '14013at7088'], '14186at7088': [0.149, 'HINT3'], '1418at7088': [0.044, '1418at7088'], '14655at7088': [-0.039, 'KRTCAP2'], '15500at7088': [0.248, 'DPY30'], '15652at7088': [-0.02, '15652at7088'], '1970at7088': [-0.102, '1970at7088'], '2121at7088': [-0.015, 'DDX10'], '2390at7088': [-0.015, '2390at7088'], '11065at7088': [0.175, 'QDPR'], '11066at7088': [0.037, 'TP53RK'], '11069at7088': [0.023, 'OVCA2'], '11070at7088': [0.027, 'DCUN1D5'], '11072at7088': [-0.072, '11072at7088'], '11080at7088': [0.028, 'HMGXB4'], '11

In [6]:
rho_df = pd.DataFrame.from_dict(rho_dict, orient="index", columns=["rho", "annotation"])
rho_df.index.name = "sseqid"   # make keys line up with top_hits.sseqid
top_hits["rho_genes"] = top_hits["sseqid"].apply(lambda x: "Yes" if x in rho_genes else "No")
top_hits["extended_rho_genes"] = top_hits["sseqid"].apply(lambda x: "Yes" if x in extended_rho_set else "No")
# Merge on sseqid
merged = top_hits.merge(rho_df, how="left", left_on="sseqid", right_index=True)
merged = merged.drop_duplicates(subset="sseqid", keep="first")

# Save
merged.to_csv("/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/24.Transcriptome_comparisons/1.Baker_et_al_2009_Gene_expression_during/top_hits__with_rho_blast_results_Diapause induced gene table.csv", sep=",", index=False)
print(merged)

          qseqid       sseqid  pident  length  qlen  slen         evalue  \
0    FBgn0000056  10359at7088  27.354     223   272   253   2.300000e-08   
1    FBgn0000092  10750at7088  27.511     229   260   234   2.660000e-25   
2    FBgn0000299    964at7088  42.325    1798  1779  1758   0.000000e+00   
3    FBgn0000570   7277at7088  56.948     367   483   359  1.580000e-137   
4    FBgn0000635    496at7088  23.944     497   873   897   6.730000e-18   
..           ...          ...     ...     ...   ...   ...            ...   
259  FBgn0086347    116at7088  36.649     734  1011  2108  5.420000e-143   
262  FBgn0250876   1461at7088  42.857    1078  1277  1081   0.000000e+00   
263  FBgn0259149   4371at7088  28.641     412   521   725   8.950000e-48   
266  FBgn0259211   1253at7088  54.973     744  1333   701   0.000000e+00   
267  FBgn0259680   1372at7088  70.686     423   671  1333   0.000000e+00   

     bitscore        qcov rho_genes extended_rho_genes    rho   annotation  
0        5